In [ ]:
# Installing dependencies
!pip install llama-index --upgrade
!pip install llama-index-llms-groq
!pip install llama-index-embeddings-huggingface
!pip install llama-index-llms-huggingface
!pip install llama-index-llms-huggingface-api

In [ ]:
# Imports
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext, PromptTemplate, Settings
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.llms.groq import Groq
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.chat_engine import CondensePlusContextChatEngine
import os

llm = Groq(model="llama3-70b-8192", api_key=os.environ.get("GROQ_API_KEY1"))

embed_model = HuggingFaceEmbedding(model_name="mixedbread-ai/mxbai-embed-large-v1")

Settings.llm = llm
Settings.embed_model = embed_model

# Corrected the path to a directory instead of a file
de_tools_blog = SimpleDirectoryReader("./", required_exts=[".pdf"]).load_data()

# print(de_tools_blog)

index = VectorStoreIndex.from_documents(de_tools_blog)
query_engine = index.as_query_engine(similarity_top_k=3)

response = query_engine.query("How many concepts are used?")
print(response)

memory = ChatMemoryBuffer.from_defaults(token_limit=3900)

chat_engine = CondensePlusContextChatEngine.from_defaults(    
   index.as_retriever(),    
   memory=memory,    
   llm=llm
)

response = chat_engine.chat(    
   "What is Yulu company all about?"
)
print(str(response))